# `geom_band()`

In [1]:
%useLatestDescriptors
%use dataframe

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.0.0.0-SNAPSHOT. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.4.0.

## Default

In [3]:
letsPlot() + geomBand(xmin = -1, xmax = 1)

## Both vertical and horizontal

In [4]:
letsPlot() +
    geomBand { xmin = listOf(-1.5, .5); xmax = listOf(-.5, 1.5) } +
    geomBand { ymin = listOf(-1.5, .5); ymax = listOf(-.5, 1.5) }

## Additional parameters

In [5]:
letsPlot() +
    geomBand(size = 3, alpha = 0.5, linetype = "dashed", showLegend = false)
        { xmin = listOf(-1, 0); xmax = listOf(0, 1); fill = listOf("a", "b") }

In [6]:
// stat

letsPlot(mapOf(
    "x" to listOf("a", "a", "a", "b", "b", "b"),
    "y" to listOf(1, 2, 3, 4, 5, 6)
)) { x = "x"; y = "y" } + geomBand(stat = Stat.boxplot()) { fill = "x" }

In [7]:
// position

letsPlot() +
    geomBand(position = positionNudge(x = 1)) { xmin = listOf(-1); xmax = listOf(1) } +
    xlim(listOf(-1, 3))

In [8]:
// sampling

val n = 20
letsPlot() +
    geomBand(size = 0, sampling = samplingRandom(n - 1, seed = 42)) {
        xmin = (0 until n).toList()
        xmax = (1..n).toList()
        fill = (0 until n).toList()
    }

In [9]:
// colorBy/fillBy

letsPlot() +
    geomBand(colorBy = "paint_a", fillBy = "paint_b", size = 1.5) {
        xmin = listOf(-3, 1)
        xmax = listOf(-1, 3)
        paint_a = listOf("a", "b")
        paint_b = listOf("A", "B")
    } +
    scaleBrewer("paint_a", palette = "Set2") +
    scaleBrewer("paint_b", palette = "Pastel2")

## Tooltips

In [10]:
val dataMap = mapOf(
    "xstart" to listOf( 0,  2,  3,    6, null),
    "xend" to   listOf( 1,  4,  5,    7, null),
    "yFrom" to  listOf(-1, -4, -5, null,   -9),
    "yTo" to    listOf( 0, -2, -3, null,   -8),
    "color" to  listOf("black", "red", "blue", "green", "yellow"),
)

val p1 = letsPlot(dataMap) { xmin = "xstart"; xmax = "xend" }
val p2 = letsPlot(dataMap) { ymin = "yFrom"; ymax = "yTo" }

gggrid(listOf(
    p1 + geomBand(alpha = 0.2) { fill = "color" } + scaleFillIdentity(),
    p2 + geomBand(alpha = 0.2) { fill = "color" } + scaleFillIdentity(),
    p1 + geomBand(tooltips = tooltipsNone),
    p1 + geomBand(tooltips = layerTooltips().line("min|^xmin").line("max|^xmax").line("color|@color")),
), ncol = 2)

## Different coordinate systems

In [11]:
val limits = Pair(-4.25, 4.25)

val p = letsPlot(mapOf(
    "xmin" to listOf(-4, -2.5, -1,  1.5,    3),
    "xmax" to listOf(-3, -1.5,  1,  2.5,    4),
    "ymin" to listOf(-4, -1,    3, null, null),
    "ymax" to listOf(-3,  1,    4, null, null),
)) +
    geomBand(alpha = 0.2, size = 0, fill = "red") { xmin = "xmin"; xmax = "xmax" } +
    geomBand(alpha = 0.2, size = 0, fill = "blue") { ymin = "ymin"; ymax = "ymax" }

gggrid(listOf(
    p + ggtitle("Default coord"),
    p + coordFlip(xlim = limits, ylim = limits) + ggtitle("coordFlip()"),
    p + coordFixed(xlim = limits, ylim = limits) + ggtitle("coordFixed()"),
    p + coordPolar(xlim = limits, ylim = limits) + ggtitle("coordPolar()")
), ncol = 2)

## More complex example

In [12]:
// See: https://gadflyjl.org/stable/gallery/geometries/#[Geom.band](@ref),-[Geom.hband](@ref),-[Geom.vband](@ref)

import kotlinx.datetime.*

fun DataFrame<*>.parseDates(column: String): DataFrame<*> {
    return this.convert(column).toLocalDateTime()
               .convert(column).with { (it as LocalDateTime).toInstant(TimeZone.currentSystemDefault()).toEpochMilliseconds() }
}

val presidentialDf = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/presidential.csv")
    .parseDates("start").parseDates("end")
val economicsDf = DataFrame.readCSV("https://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/economics.csv")
    .parseDates("date")

letsPlot() +
    geomBand(data = presidentialDf.toMap(), size = 0, alpha = 0.4) { xmin = "start"; xmax = "end"; fill = "party" } +
    geomLine(data = economicsDf.toMap()) { x = "date"; y = "unemploy" } +
    scaleXDateTime() + scaleYContinuous(trans = "log10") +
    ggsize(1000, 200)